In [13]:
import numpy as np
import tensorflow as tf
import keras
from tqdm import tqdm
import keras.backend as K

import os
import time
from datetime import datetime
import nets
from Data import datagen

In [14]:
ENVIRON = 'snuh'

In [15]:
dim_HT1D = (6000,1) if ENVIRON == 'snuh' else (3000,1)
n_classes = 5 if ENVIRON == 'snuh' else 6
bs = 64
BASE_LEARNING_RATE = 1e-3
model_flag = 6
data_ratio = 1.0

In [16]:
import importlib 
importlib.reload(nets)  # Python 3.4+
model = nets.Conv1D_SPP_functional(n_classes=n_classes)

In [17]:
x = np.random.random((1,) + dim_HT1D)
x = tf.convert_to_tensor(x)
x = tf.expand_dims(x, axis=-1)
print(model(x))
print(model.name)

tf.Tensor([[0.19483492 0.22842051 0.17717429 0.19882071 0.20074958]], shape=(1, 5), dtype=float32)
conv1d_spp


In [18]:
model(x)

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[0.19483492, 0.22842051, 0.17717429, 0.19882071, 0.20074958]],
      dtype=float32)>

In [19]:
from tensorflow.keras.layers \
    import BatchNormalization, Conv1D, Conv2D, ReLU, Input, Dense, Flatten, RepeatVector, Reshape, Dropout, add,\
        MaxPool1D, MaxPool2D, GlobalAveragePooling1D, GlobalMaxPooling1D, LSTM, LayerNormalization, Conv1DTranspose, Lambda, \
        GlobalAveragePooling2D, Concatenate, Conv2DTranspose

In [20]:
def match_annotations_npy(dirname, filepath):
    filename = os.path.basename(filepath)
    if ENVIRON == 'snuh':
        search_filename = filename.split('.')[0]
    else:
        search_filename = filename.split('-')[0][:-2]
        
    file_list = os.listdir(dirname)
    filenames = [f for f in file_list if search_filename in f if f.endswith('.npy')]
    return filenames

In [41]:
if ENVIRON == 'snuh':
    PROCESSED_DATA_PATH = os.path.join('/tf','00_data','kdpark')
    save_signals_path = os.path.join(PROCESSED_DATA_PATH,'signals_filtered_C3A2')
    save_annotations_path = os.path.join(PROCESSED_DATA_PATH,'sleep_edf','all_channels','annotations')
    list_files = [os.path.join(save_signals_path, f) for f in os.listdir(save_signals_path) if f.endswith('.npy')]
    train_test_split = 0.7
    split_cnt = int(train_test_split * len(list_files))

    list_files_train = []
    list_files_test = []

    list_ann_files_train = []
    list_ann_files_test = []

    list_files_train = np.random.choice(list_files[:split_cnt], int(float(data_ratio) * split_cnt), replace=False)
    list_files_train = list_files_train.tolist()
    for f in list_files_train:
        ann_file = match_annotations_npy(save_annotations_path, f)
        list_ann_files_train.append(os.path.join(save_annotations_path, ann_file[0]))

    list_files_test += list_files[split_cnt:]

    for f in list_files[split_cnt:]:
        ann_file = match_annotations_npy(save_annotations_path, f)
        list_ann_files_test.append(os.path.join(save_annotations_path, ann_file[0]))

else:
    PROCESSED_DATA_PATH = os.path.join('/home','aiot','data','origin_npy')
    save_signals_path_SC = os.path.join(PROCESSED_DATA_PATH,'signals_SC_filtered')
    save_annotations_path_SC = os.path.join(PROCESSED_DATA_PATH,'annotations_SC')
    save_signals_path_ST = os.path.join(PROCESSED_DATA_PATH,'signals_ST_filtered')
    save_annotations_path_ST = os.path.join(PROCESSED_DATA_PATH,'annotations_ST')

    list_files_SC = [os.path.join(save_signals_path_SC, f) for f in os.listdir(save_signals_path_SC) if f.endswith('.npy')]
    list_files_ST = [os.path.join(save_signals_path_ST, f) for f in os.listdir(save_signals_path_ST) if f.endswith('.npy')]

    train_test_split = 0.7
    split_cnt_SC = int(train_test_split * len(list_files_SC))
    split_cnt_ST = int(train_test_split * len(list_files_ST))

    SC_ST = [x for x in FLAGS.include_type.split(',')]
    include_SC = 'SC' in SC_ST
    include_ST = 'ST' in SC_ST

    list_files_train = []
    list_files_test = []

    list_ann_files_train = []
    list_ann_files_test = []

    if include_SC:
        list_files_SC_train = np.random.choice(list_files_SC[:split_cnt_SC], int(float(data_ratio) * split_cnt_SC), replace=False)
        list_files_train += list_files_SC_train.tolist()
        for f in list_files_SC_train:
            ann_file = match_annotations_npy(save_annotations_path_SC, f)
            list_ann_files_train.append(os.path.join(save_annotations_path_SC, ann_file[0]))

        list_files_test += list_files_SC[split_cnt_SC:]

        for f in list_files_SC[split_cnt_SC:]:
            ann_file = match_annotations_npy(save_annotations_path_SC, f)
            list_ann_files_test.append(os.path.join(save_annotations_path_SC, ann_file[0]))

    if include_ST:
        list_files_ST_train = np.random.choice(list_files_ST[:split_cnt_ST], int(float(data_ratio) * split_cnt_ST), replace=False)
        list_files_train += list_files_ST_train.tolist()
        for f in list_files_ST_train:
            ann_file = match_annotations_npy(save_annotations_path_ST, f)
            list_ann_files_train.append(os.path.join(save_annotations_path_ST, ann_file[0]))

        list_files_test += list_files_ST[split_cnt_ST:]
        for f in list_files_ST[split_cnt_ST:]:
            ann_file = match_annotations_npy(save_annotations_path_ST, f)
            list_ann_files_test.append(os.path.join(save_annotations_path_ST, ann_file[0]))

train_generator = datagen.DataGenerator(list_files_train, list_ann_files_train, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=True, balanced_sampling=False)
test_generator = datagen.DataGenerator(list_files_test, list_ann_files_test, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=False, balanced_sampling=False)

In [22]:
import tensorflow_model_optimization as tfmot
import tempfile

In [23]:
checkpoint_path = 'tf_ckpt/ckpt_conv1d_spp_func/ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [24]:
class CustomLearningRateScheduler(keras.callbacks.Callback):
    """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # Call schedule function to get the scheduled learning rate.
        scheduled_lr = self.schedule(epoch, lr)
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, scheduled_lr))


LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (10, 5e-4),
    (20, 1e-4),
    (30, 5e-5),
    (40, 1e-5)    
]


def lr_schedule(epoch, lr):
    """Helper function to retrieve the scheduled learning rate based on epoch."""
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][0]:
            return LR_SCHEDULE[i][1]
    return lr


In [25]:
model.compile(tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
model.fit(train_generator, epochs=50, callbacks=[cp_callback, CustomLearningRateScheduler(lr_schedule)])


Epoch 00000: Learning rate is 0.0010.
Epoch 1/50
7276/7276 [==============================] - ETA: 0s - loss: 0.8223 - accuracy: 0.7011
Epoch 1: saving model to tf_ckpt/ckpt_conv1d_spp_func/ckpt
7276/7276 [==============================] - 456s 62ms/step - loss: 0.8223 - accuracy: 0.7011

Epoch 00001: Learning rate is 0.0010.
Epoch 2/50
7276/7276 [==============================] - ETA: 0s - loss: 0.6366 - accuracy: 0.7574
Epoch 2: saving model to tf_ckpt/ckpt_conv1d_spp_func/ckpt
7276/7276 [==============================] - 456s 63ms/step - loss: 0.6366 - accuracy: 0.7574

Epoch 00002: Learning rate is 0.0010.
Epoch 3/50
7276/7276 [==============================] - ETA: 0s - loss: 0.5861 - accuracy: 0.7733
Epoch 3: saving model to tf_ckpt/ckpt_conv1d_spp_func/ckpt
7276/7276 [==============================] - 453s 62ms/step - loss: 0.5861 - accuracy: 0.7733

Epoch 00003: Learning rate is 0.0010.
Epoch 4/50
7276/7276 [==============================] - ETA: 0s - loss: 0.5549 - accuracy: 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7276/7276 [==============================] - ETA: 0s - loss: 0.3558 - accuracy: 0.8596
Epoch 22: saving model to tf_ckpt/ckpt_conv1d_spp_func/ckpt
7276/7276 [==============================] - 457s 63ms/step - loss: 0.3558 - accuracy: 0.8596

Epoch 00022: Learning rate is 0.0001.
Epoch 23/50
 457/7276 [>.............................] - ETA: 7:09 - loss: 0.3526 - accuracy: 0.8621

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



5386/7276 [=====================>........] - ETA: 1:58 - loss: 0.3517 - accuracy: 0.8614

...

5201/7276 [====================>.........] - ETA: 2:11 - loss: 0.3039 - accuracy: 0.8796

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7276/7276 [==============================] - ETA: 0s - loss: 0.3050 - accuracy: 0.8795
Epoch 48: saving model to tf_ckpt/ckpt_conv1d_spp_func/ckpt
7276/7276 [==============================] - 461s 63ms/step - loss: 0.3050 - accuracy: 0.8795

Epoch 00048: Learning rate is 0.0000.
Epoch 49/50
2808/7276 [==========>...................] - ETA: 4:40 - loss: 0.3017 - accuracy: 0.8804

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



5098/7276 [====================>.........] - ETA: 2:17 - loss: 0.3023 - accuracy: 0.8806

In [ ]:
# test_generator = datagen.DataGenerator(list_files_test, list_ann_files_test, 
#                           batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=False, balanced_sampling=False)

In [42]:
model.evaluate(test_generator)

3071/3071 [==============================] - 142s 46ms/step - loss: 0.6828 - accuracy: 0.7731


[0.6828160881996155, 0.7731318473815918]

In [ ]:
quant_aware_model = tfmot.quantization.keras.quantize_model(model)
quant_aware_model.summary()

In [43]:
quant_aware_model.compile(tf.keras.optimizers.Adam(learning_rate=1e-5),
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                          metrics=['accuracy'])

In [44]:
checkpoint_path = 'tf_ckpt/ckpt_conv1d_spp_quant/ckpt'
os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback2 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [45]:
quant_aware_model.fit(train_generator, epochs=10, callbacks=[cp_callback2])

Epoch 1/10
7276/7276 [==============================] - ETA: 0s - loss: 0.4322 - accuracy: 0.8292
Epoch 1: saving model to tf_ckpt/ckpt_conv1d_spp_quant/ckpt
7276/7276 [==============================] - 918s 126ms/step - loss: 0.4322 - accuracy: 0.8292
Epoch 2/10
7276/7276 [==============================] - ETA: 0s - loss: 0.4296 - accuracy: 0.8306
Epoch 2: saving model to tf_ckpt/ckpt_conv1d_spp_quant/ckpt
7276/7276 [==============================] - 912s 125ms/step - loss: 0.4296 - accuracy: 0.8306
Epoch 3/10
7276/7276 [==============================] - ETA: 0s - loss: 0.4287 - accuracy: 0.8305
Epoch 3: saving model to tf_ckpt/ckpt_conv1d_spp_quant/ckpt
7276/7276 [==============================] - 923s 127ms/step - loss: 0.4287 - accuracy: 0.8305
Epoch 4/10
7276/7276 [==============================] - ETA: 0s - loss: 0.4265 - accuracy: 0.8315
Epoch 4: saving model to tf_ckpt/ckpt_conv1d_spp_quant/ckpt
7276/7276 [==============================] - 920s 126ms/step - loss: 0.4265 - accur

In [33]:
quant_aware_model.evaluate(test_generator)

3000/3071 [============================>.] - ETA: 5s - loss: 0.6657 - accuracy: 0.7587

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [46]:
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp7w830owr/assets


INFO:tensorflow:Assets written to: /tmp/tmp7w830owr/assets
/usr/local/lib/python3.8/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [47]:
with open(os.path.join('tflite', model.name + '_quant.tflite'), 'wb') as f:
    f.write(quantized_tflite_model) 

In [48]:
tflite_name = 'conv1d_spp_quant.tflite'
interpreter = tf.lite.Interpreter(model_path=os.path.join('tflite', tflite_name))
interpreter.allocate_tensors()
tflite_input_details = interpreter.get_input_details()
tflite_output_details = interpreter.get_output_details()

In [49]:
# Recreate test_generator with batch_size=1
test_generator = datagen.DataGenerator(list_files_test, list_ann_files_test, 
                          batch_size=1, dim=dim_HT1D, n_classes=n_classes, shuffle=False, balanced_sampling=False)

In [51]:
def test_tflite(x, y):
    interpreter.set_tensor(tflite_input_details[0]['index'], x)
    interpreter.invoke()
    y_pred = interpreter.get_tensor(tflite_output_details[0]['index'])
    return tf.convert_to_tensor(y_pred)
    


correct, total_cnt, total_loss = 0.0, 0.0, 0.0
confusion_matrix = np.zeros((n_classes,n_classes))
start_time = time.time()
for idx, (x, y) in enumerate(test_generator):
    # y_pred = model(x, training=False)
    x = tf.expand_dims(x, -1)
    y_pred = test_tflite(x, y)    
    y_pred_cls = tf.math.argmax(y_pred, axis=-1)
    correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls, y), tf.float32))
    total_cnt += y_pred.shape[0]
    y = tf.cast(y, dtype=tf.int32)    
    test_acc = correct / total_cnt    
    if (idx + 1) % 10 == 0 or idx+1 == len(test_generator):
        elapsed_time = time.time() - start_time
        eta = elapsed_time * len(test_generator) / (idx+1) - elapsed_time
        print("[%d / %d], test accuracy: %.3f, elapsed_time: %3.f, ETA: %3.f"%
            (idx+1, len(test_generator), test_acc, elapsed_time, eta),end='\r', flush=True)
    
    for i in range(n_classes):
        for j in range(n_classes):
            confusion_matrix[i,j] += np.sum((y_pred_cls.numpy()==i) * (y.numpy()==j))

IndexError: index 281 is out of bounds for axis 0 with size 281

In [ ]:
@tf.function
def test_step(x, y):
    y_pred = model(x, training=False)
    return y_pred

def test_tflite(x, y):
    interpreter.set_tensor(tflite_input_details[0]['index'], x)
    interpreter.invoke()
    y_pred = interpreter.get_tensor(tflite_output_details[0]['index'])
    return tf.convert_to_tensor(y_pred)
    


correct, total_cnt, total_loss = 0.0, 0.0, 0.0
confusion_matrix = np.zeros((n_classes,n_classes))
for idx, (x, y) in enumerate(test_generator):
    # y_pred = model(x, training=False)
    x = tf.expand_dims(x, -1)
#     y_pred = test_tflite(x, y)    
    y_pred = test_step(x, y)
    y_pred_cls = tf.math.argmax(y_pred, axis=-1)
    correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls, y), tf.float32))
    total_cnt += y_pred.shape[0]
    y = tf.cast(y, dtype=tf.int32)    
    test_acc = correct / total_cnt    
    if (idx + 1) % 10 == 0 or idx+1 == len(test_generator):
        print("[%d / %d], test accuracy: %.3f"%
            (idx+1, len(test_generator), test_acc),end='\r', flush=True)
    
    for i in range(n_classes):
        for j in range(n_classes):
            confusion_matrix[i,j] += np.sum((y_pred_cls.numpy()==i) * (y.numpy()==j))

In [13]:
def representative_data_gen():
    for idx, (x, y) in enumerate(train_generator):            
        if idx >= 10000: break
        print(f'Processing {idx}th batch')
        yield [tf.expand_dims(x, -1)]

In [14]:
# TFlite conversion
def tflite_convert(model, out_dir):
    # A generator that provides a representative dataset

    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    # This enables quantization
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    # This sets the representative dataset for quantization
    converter.representative_dataset = representative_data_gen
    # This ensures that if any ops can't be quantized, the converter throws an error
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    # For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
    converter.target_spec.supported_types = [tf.int8]
    # These set the input and output tensors to uint8 (added in r2.3)
    converter.inference_input_type = tf.float32
    converter.inference_output_type = tf.float32
    tflite_model = converter.convert()

    with open(os.path.join(out_dir, model.name + '_quant.tflite'), 'wb') as f:
        f.write(tflite_model)        


In [15]:
tflite_convert(model, 'tflite')

INFO:tensorflow:Assets written to: /tmp/tmphc2bk3ig/assets


INFO:tensorflow:Assets written to: /tmp/tmphc2bk3ig/assets
/usr/local/lib/python3.8/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


Processing 0th batch
Processing 1th batch
Processing 2th batch
Processing 3th batch
Processing 4th batch
Processing 5th batch
Processing 6th batch
Processing 7th batch
Processing 8th batch
Processing 9th batch
Processing 10th batch
Processing 11th batch
Processing 12th batch
Processing 13th batch
Processing 14th batch
Processing 15th batch
Processing 16th batch
Processing 17th batch
Processing 18th batch
Processing 19th batch
Processing 20th batch
Processing 21th batch
Processing 22th batch
Processing 23th batch
Processing 24th batch
Processing 25th batch
Processing 26th batch
Processing 27th batch
Processing 28th batch
Processing 29th batch
Processing 30th batch
Processing 31th batch
Processing 32th batch
Processing 33th batch
Processing 34th batch
Processing 35th batch
Processing 36th batch
Processing 37th batch
Processing 38th batch
Processing 39th batch
Processing 40th batch
Processing 41th batch
Processing 42th batch
Processing 43th batch
Processing 44th batch
Processing 45th batc

Processing 362th batch
Processing 363th batch
Processing 364th batch
Processing 365th batch
Processing 366th batch
Processing 367th batch
Processing 368th batch
Processing 369th batch
Processing 370th batch
Processing 371th batch
Processing 372th batch
Processing 373th batch
Processing 374th batch
Processing 375th batch
Processing 376th batch
Processing 377th batch
Processing 378th batch
Processing 379th batch
Processing 380th batch
Processing 381th batch
Processing 382th batch
Processing 383th batch
Processing 384th batch
Processing 385th batch
Processing 386th batch
Processing 387th batch
Processing 388th batch
Processing 389th batch
Processing 390th batch
Processing 391th batch
Processing 392th batch
Processing 393th batch
Processing 394th batch
Processing 395th batch
Processing 396th batch
Processing 397th batch
Processing 398th batch
Processing 399th batch
Processing 400th batch
Processing 401th batch
Processing 402th batch
Processing 403th batch
Processing 404th batch
Processing 

Processing 722th batch
Processing 723th batch
Processing 724th batch
Processing 725th batch
Processing 726th batch
Processing 727th batch
Processing 728th batch
Processing 729th batch
Processing 730th batch
Processing 731th batch
Processing 732th batch
Processing 733th batch
Processing 734th batch
Processing 735th batch
Processing 736th batch
Processing 737th batch
Processing 738th batch
Processing 739th batch
Processing 740th batch
Processing 741th batch
Processing 742th batch
Processing 743th batch
Processing 744th batch
Processing 745th batch
Processing 746th batch
Processing 747th batch
Processing 748th batch
Processing 749th batch
Processing 750th batch
Processing 751th batch
Processing 752th batch
Processing 753th batch
Processing 754th batch
Processing 755th batch
Processing 756th batch
Processing 757th batch
Processing 758th batch
Processing 759th batch
Processing 760th batch
Processing 761th batch
Processing 762th batch
Processing 763th batch
Processing 764th batch
Processing 

...

Processing 8297th batch
Processing 8298th batch
Processing 8299th batch
Processing 8300th batch
Processing 8301th batch
Processing 8302th batch
Processing 8303th batch
Processing 8304th batch
Processing 8305th batch
Processing 8306th batch
Processing 8307th batch
Processing 8308th batch
Processing 8309th batch
Processing 8310th batch
Processing 8311th batch
Processing 8312th batch
Processing 8313th batch
Processing 8314th batch
Processing 8315th batch
Processing 8316th batch
Processing 8317th batch
Processing 8318th batch
Processing 8319th batch
Processing 8320th batch
Processing 8321th batch
Processing 8322th batch
Processing 8323th batch
Processing 8324th batch
Processing 8325th batch
Processing 8326th batch
Processing 8327th batch
Processing 8328th batch
Processing 8329th batch
Processing 8330th batch
Processing 8331th batch
Processing 8332th batch
Processing 8333th batch
Processing 8334th batch
Processing 8335th batch
Processing 8336th batch
Processing 8337th batch
Processing 8338t

Processing 8639th batch
Processing 8640th batch
Processing 8641th batch
Processing 8642th batch
Processing 8643th batch
Processing 8644th batch
Processing 8645th batch
Processing 8646th batch
Processing 8647th batch
Processing 8648th batch
Processing 8649th batch
Processing 8650th batch
Processing 8651th batch
Processing 8652th batch
Processing 8653th batch
Processing 8654th batch
Processing 8655th batch
Processing 8656th batch
Processing 8657th batch
Processing 8658th batch
Processing 8659th batch
Processing 8660th batch
Processing 8661th batch
Processing 8662th batch
Processing 8663th batch
Processing 8664th batch
Processing 8665th batch
Processing 8666th batch
Processing 8667th batch
Processing 8668th batch
Processing 8669th batch
Processing 8670th batch
Processing 8671th batch
Processing 8672th batch
Processing 8673th batch
Processing 8674th batch
Processing 8675th batch
Processing 8676th batch
Processing 8677th batch
Processing 8678th batch
Processing 8679th batch
Processing 8680t

Processing 8981th batch
Processing 8982th batch
Processing 8983th batch
Processing 8984th batch
Processing 8985th batch
Processing 8986th batch
Processing 8987th batch
Processing 8988th batch
Processing 8989th batch
Processing 8990th batch
Processing 8991th batch
Processing 8992th batch
Processing 8993th batch
Processing 8994th batch
Processing 8995th batch
Processing 8996th batch
Processing 8997th batch
Processing 8998th batch
Processing 8999th batch
Processing 9000th batch
Processing 9001th batch
Processing 9002th batch
Processing 9003th batch
Processing 9004th batch
Processing 9005th batch
Processing 9006th batch
Processing 9007th batch
Processing 9008th batch
Processing 9009th batch
Processing 9010th batch
Processing 9011th batch
Processing 9012th batch
Processing 9013th batch
Processing 9014th batch
Processing 9015th batch
Processing 9016th batch
Processing 9017th batch
Processing 9018th batch
Processing 9019th batch
Processing 9020th batch
Processing 9021th batch
Processing 9022t

Processing 9325th batch
Processing 9326th batch
Processing 9327th batch
Processing 9328th batch
Processing 9329th batch
Processing 9330th batch
Processing 9331th batch
Processing 9332th batch
Processing 9333th batch
Processing 9334th batch
Processing 9335th batch
Processing 9336th batch
Processing 9337th batch
Processing 9338th batch
Processing 9339th batch
Processing 9340th batch
Processing 9341th batch
Processing 9342th batch
Processing 9343th batch
Processing 9344th batch
Processing 9345th batch
Processing 9346th batch
Processing 9347th batch
Processing 9348th batch
Processing 9349th batch
Processing 9350th batch
Processing 9351th batch
Processing 9352th batch
Processing 9353th batch
Processing 9354th batch
Processing 9355th batch
Processing 9356th batch
Processing 9357th batch
Processing 9358th batch
Processing 9359th batch
Processing 9360th batch
Processing 9361th batch
Processing 9362th batch
Processing 9363th batch
Processing 9364th batch
Processing 9365th batch
Processing 9366t

Processing 9669th batch
Processing 9670th batch
Processing 9671th batch
Processing 9672th batch
Processing 9673th batch
Processing 9674th batch
Processing 9675th batch
Processing 9676th batch
Processing 9677th batch
Processing 9678th batch
Processing 9679th batch
Processing 9680th batch
Processing 9681th batch
Processing 9682th batch
Processing 9683th batch
Processing 9684th batch
Processing 9685th batch
Processing 9686th batch
Processing 9687th batch
Processing 9688th batch
Processing 9689th batch
Processing 9690th batch
Processing 9691th batch
Processing 9692th batch
Processing 9693th batch
Processing 9694th batch
Processing 9695th batch
Processing 9696th batch
Processing 9697th batch
Processing 9698th batch
Processing 9699th batch
Processing 9700th batch
Processing 9701th batch
Processing 9702th batch
Processing 9703th batch
Processing 9704th batch
Processing 9705th batch
Processing 9706th batch
Processing 9707th batch
Processing 9708th batch
Processing 9709th batch
Processing 9710t

In [14]:
pip install -q tensorflow-model-optimization

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [57]:
interpreter = tf.lite.Interpreter(model_path=os.path.join('tflite', model.name + '_quant.tflite'))

In [65]:
input_details

[{'name': 'serving_default_input_1:0',
  'index': 0,
  'shape': array([   1, 6000,    1], dtype=int32),
  'shape_signature': array([  -1, 6000,    1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [58]:
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [59]:
x,y = next(iter(train_generator))

In [60]:
interpreter.set_tensor(input_details[0]['index'], x[:1])

In [61]:
interpreter.invoke()
interpreter.get_tensor(output_details[0]['index'])

array([[0.1640625 , 0.16015625, 0.1796875 , 0.16015625, 0.1640625 ,
        0.171875  ]], dtype=float32)

In [62]:
import numpy as np

In [63]:
x = [np.array([1,2,3]), np.array([4,5,6])]

In [64]:
np.array(x).flatten()

array([1, 2, 3, 4, 5, 6])

In [39]:
x = 1
print(x)

1
